In [37]:
from binance.client import Client
import pandas as pd
import json
import numpy as np
# from sklearn.linear_model import LinearRegression

In [38]:
from security import get_keys
binance_key = get_keys('client')
client = Client(binance_key['api_key'], binance_key['api_secret'])

correct password


In [32]:
def formatPrice(n):
	return ("-$ " if n < 0 else "$ ") + "{0:.4f} %".format(abs(n))

In [33]:
print(formatPrice(5))

$ 5.0000 %


In [34]:
def get_total_asset(coin_list, base_coin):
	total_asset = 0.0
	for coin_symbol in coin_list:
		pair = coin_symbol + base_coin
		if coin_symbol == base_coin:
			total_asset += float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free'])
			continue
		total_asset += float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free']) * float(client.get_symbol_ticker(symbol=pair)['price'])
	return total_asset

In [35]:
initial_asset = 0.0

In [41]:
    gain = 0.0
    if initial_asset == 0.0:
        initial_asset = get_total_asset(coin_list = ['BTC', 'USDT'], base_coin = 'USDT')
    else:
        current_asset = get_total_asset(coin_list = ['BTC', 'USDT'], base_coin = 'USDT')
        gain = (current_asset/initial_asset) - 1.0
        gain *= 100.0
    display =  ("-" if gain < 0 else "+") + "{0:.4f} %".format(abs(gain))
    print(display)

+0.0000 %


In [42]:
initial_asset

47.22113941

In [263]:
data = pd.read_csv("BTCUSDT.csv", header=None, usecols=[2,8])
data = data.rename(columns={2: "date", 8: "price"})
data.date = pd.to_datetime(data.date, unit='ms')


,date,price
0,2020-09-27 21:00:20.504,10721.66
1,2020-09-27 21:00:21.476,10721.27
2,2020-09-27 21:00:22.502,10721.27
3,2020-09-27 21:00:23.329,10721.27
4,2020-09-27 21:00:24.466,10720.87
...,...,...
16020,2020-09-28 14:38:30.295,10879.20
16021,2020-09-28 14:38:31.292,10879.19
16022,2020-09-28 14:38:32.265,10879.20
16023,2020-09-28 14:38:33.301,10879.20


In [54]:
pairs = 'BTCUSDT'

In [57]:
def MA(seconds):
	df = pd.read_csv(pairs+'.csv', header=None, usecols=[2,8])
	df = df.rename(columns={2: "date", 8: "price"})
	df.date = pd.to_datetime(df.date, unit='ms')
	start_time = df.date.iloc[-1] - pd.Timedelta(seconds=seconds)
	# return df.loc[df.date >= start_time]
	# start_time = status.date.iloc[-1] - pd.Timedelta(seconds=seconds)
	df = df.loc[df.date >= start_time]
	return df.price.mean()

In [78]:
base = MA(297)
x = (MA(33) / base) -1.0
x * 100
print(x)
x = (MA(99) / base) -1.0
x * 100
print(x)

0.00032959042534774774
0.00039512622225723604


In [279]:
price = {'BTCUSDT': csvdata, 'error':False}

In [282]:
def MA(sec):
	df = price['BTCUSDT']
	start_time = df.date.iloc[-1] - pd.Timedelta(seconds=sec)
	df = df.loc[df.date >= start_time]
	return df.price.mean()

In [13]:
for x in range(10):
    print(MA(x*30))

10903.69
10908.35
10911.61
10912.4
10913.28
10913.49
10913.49
10913.49
10913.49
10913.49


In [280]:
x = MA(30) / MA(900)
x -= 1.0
x *= 100.0
print(x, '%')

-0.07237538915328967 %


In [213]:
def buyAmount(coin, pair):
    balanceBuy = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    close = float(client.get_symbol_ticker(symbol=pair)['price'])
    maxBuy = round(balanceBuy / close * .995, 5)
    return maxBuy
    
def sellAmount(coin):
    balanceSell = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    maxSell = round(balanceSell * .995, 5)
    return maxSell

In [297]:
buyAmount('USDT', 'BTCUSDT')

0.0043

In [296]:
sellAmount('BTC')

2e-05

In [299]:
maxbuy = buyAmount('USDT', 'BTCUSDT')
client.create_test_order(symbol='BTCUSDT', side='BUY', type='MARKET', quantity=0.00434)

{}

In [304]:
maxsell = sellAmount('BTC')
client.create_test_order(symbol='BTCUSDT', side='SELL', type='MARKET', quantity=0.00434)

{}

In [307]:
maxbuy = buyAmount('USDT', 'BTCUSDT')
client.create_order(symbol='BTCUSDT', side='BUY', type='MARKET', quantity=0.00433)

{'symbol': 'BTCUSDT',
 'orderId': 3293382693,
 'orderListId': -1,
 'clientOrderId': 'ReO8ipT9hhGfo0ZrDd9UtS',
 'transactTime': 1601327744874,
 'price': '0.00000000',
 'origQty': '0.00431000',
 'executedQty': '0.00431000',
 'cummulativeQuoteQty': '46.77522320',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '10852.72000000',
   'qty': '0.00431000',
   'commission': '0.00127469',
   'commissionAsset': 'BNB',
   'tradeId': 424641742}]}

In [90]:
# maxsell = sellAmount('BTC')
client.create_order(symbol='BTCUSDT', side='SELL', type='MARKET', quantity=0.00433)

{'symbol': 'BTCUSDT',
 'orderId': 3298837353,
 'orderListId': -1,
 'clientOrderId': 'lhQjJ0sxtBKS2R8wlwIrJK',
 'transactTime': 1601422407138,
 'price': '0.00000000',
 'origQty': '0.00433000',
 'executedQty': '0.00433000',
 'cummulativeQuoteQty': '46.96569140',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '10846.58000000',
   'qty': '0.00433000',
   'commission': '0.00121761',
   'commissionAsset': 'BNB',
   'tradeId': 425308268}]}

In [48]:
"%s".format('hello')

'%s'

In [79]:
def get_total_asset(coin_list, base_coin):
	total_asset = 0.0
	for coin_symbol in coin_list:
		pair = coin_symbol + base_coin
		if coin_symbol == base_coin:
			total_asset += float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free'])
			continue
		total_asset += float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free']) * float(client.get_symbol_ticker(symbol=pair)['price'])
	return total_asset

In [88]:
get_total_asset(coin_list = ['BTC', 'BNB', 'USDT'], base_coin = 'USDT')

48.483802211725